##Using HackaMol to calculate the Ramachandran Plot

In [1]:
use Modern::Perl;
use HackaMol;

### Load a molecule within multiple chains from the pdbid

In [2]:
my $pdbid    = '1axc';
my $mol      = HackaMol->new->pdbid_mol($pdbid);
my @bb_atoms = $mol->select_atoms(sub{
                                        $_ -> name eq 'C' or
                                        $_ -> name eq 'CA' or
                                        $_ -> name eq 'N'
                                     });
                                     

Warning: 1axc.pdb exists, set self->overwrite(1) to overwrite at /Users/demianriccardi/perl5/perlbrew/perls/perl-5.20.2t/lib/site_perl/5.20.2/HackaMol.pm line 26.


## Create groups of atoms according to their chains

In [3]:
my @chains = HackaMol->new->group_by_atom_attr('chain',@bb_atoms);
my $bb_mol = HackaMol::Molecule->new(groups => [@chains]);
$bb_mol->print_pdb ("$pdbid\_backbone.pdb");1

1


Warning: overwrite 1axc_backbone.pdb at reply input line 3.


##Construct the backbone dihedrals for each chain
The backbone atoms are ordered in repeats {N, CA, C}. Walking down the backbone: N0-CA0-C0-N1-CA1-C1-N2-CA2-C2 ...there will be dihedrals about the CA0-C0(PSI), C0-N1(OMEGA), N1-CA1 (PHI), CA1-C1 (PSI), ... bonds.  Since there are no bonds between chains, they must be treated separately.

In [5]:
my @dihedrals;
foreach my $chain ($bb_mol->all_groups){
    push @dihedrals, HackaMol->new->build_dihedrals($chain->all_atoms);
}